In [0]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import RandomOverSampler as ROS
from imblearn.over_sampling import SMOTE as SMO
from google.colab import drive
drive.mount('/content/gdrive')


/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
train =pd.read_csv('/content/gdrive/My Drive/ML/Training_x_ad.csv',sep='\t', encoding='utf-8')
test = pd.read_csv('/content/gdrive/My Drive/ML/Testing_x_ad.csv',sep='\t', encoding='utf-8')
data = pd.concat([train,test],axis = 0)
data = data.reset_index()
data = data.drop(columns = ['index'])
data = data[data['ArrDel15']==1]
data = data.reset_index()
data = data.drop(columns = ['index'])
data = data.drop(columns = ['Unnamed: 0'])
x_tr = ['Quarter', 'Month', 'DayofMonth', 'DayOfWeek','CRSDepTime',
        'Origin',  'Dest','year','DepDelay',
       'DewPointC', 'DewPointF', 'FeelsLikeC', 'FeelsLikeF', 'HeatIndexC',
       'HeatIndexF', 'WindChillC', 'WindChillF', 'WindGustKmph',
       'WindGustMiles', 'cloudcover', 'humidity', 'precipMM', 'pressure',
       'tempC', 'tempF', 'time', 'visibility', 'weatherCode', 'winddir16Point',
       'winddirDegree', 'windspeedKmph', 'windspeedMiles']
X= data[x_tr]
def func(row):
    if(row['year']==2016):
      row['year'] = 1  # lable encoding the year values
    else:
      row['year'] = 2  
    return row
X= X.apply(func, axis=1)
x_val = X.iloc[:,:].values
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
enc  = LabelEncoder()
enc.fit(x_val[:,6].reshape(-1, 1) )
c1=['d1']
temp = pd.DataFrame(enc.transform(x_val[:,5]),columns = c1)
X = pd.concat([X,temp],axis=1)
X = X.drop(columns = ['Origin'])
c2=['a1']
temp = pd.DataFrame(enc.transform(x_val[:,6]),columns = c2)
X = pd.concat([X,temp],axis=1)
X = X.drop(columns = ['Dest'])
X = X.drop(columns = ['winddir16Point'])
X = X.drop(columns = ['DewPointF','FeelsLikeC','FeelsLikeF','HeatIndexF','WindChillF','WindGustMiles','tempF','windspeedMiles','Quarter','time'])
x_val = X.iloc[:,:].values
Y = pd.DataFrame(data['ArrDelayMinutes'])
y_val = Y.iloc[:,:].values
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(x_val, y_val, test_size=0.33, random_state=42)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_train,y_train)
for i in zip(list(reg.coef_)[0],list(X.keys())):
  print(i)
print(reg.intercept_)

(-0.006812083381533702, 'Month')
(-0.0034789018525027138, 'DayofMonth')
(-0.05507417836949929, 'DayOfWeek')
(-0.0040481825908498395, 'CRSDepTime')
(0.5690953027775202, 'year')
(0.9136164024656619, 'DepDelay')
(0.05800338363798834, 'DewPointC')
(0.42370404675026996, 'HeatIndexC')
(-0.30834591595984, 'WindChillC')
(0.07259357005578877, 'WindGustKmph')
(0.010111906112561632, 'cloudcover')
(-0.02269898060895811, 'humidity')
(0.15146065268810066, 'precipMM')
(-0.10704083520434089, 'pressure')
(-0.14266929437764336, 'tempC')
(-0.059634831690143, 'visibility')
(0.004481297733050837, 'weatherCode')
(-0.0015015805925813717, 'winddirDegree')
(-0.06081045967945053, 'windspeedKmph')
(-0.06274468722940482, 'd1')
(0.18954954367587437, 'a1')
[123.22266788]


In [0]:
x_val1 = X[y_val>2000].iloc[:,:].values
y_val1 = Y[y_val>2000].iloc[:,:].values

In [0]:
from sklearn.metrics import mean_squared_error
import math
mse = mean_squared_error(y_val1, reg.predict(x_val1))
print("MSE: %.4f" % mse)
print("RMSE: %.4f" %math.sqrt(mse))
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_val1, reg.predict(x_val1))
print("MAE: %.4f" % mae)

MSE: 26194.1169
RMSE: 161.8460
MAE: 161.7119


In [0]:
x_val2 = X[y_val>1000].iloc[:,:].values
y_val2 = Y[y_val>1000].iloc[:,:].values

In [0]:
from sklearn.metrics import mean_squared_error
import math
mse = mean_squared_error(y_val2, reg.predict(x_val2))
print("MSE: %.4f" % mse)
print("RMSE: %.4f" %math.sqrt(mse))
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_val2, reg.predict(x_val2))
print("MAE: %.4f" % mae)

MSE: 9286.1945
RMSE: 96.3649
MAE: 92.4357


In [0]:
x_val3 = X[y_val<200].iloc[:,:].values
y_val3 = Y[y_val<200].iloc[:,:].values

In [0]:
from sklearn.metrics import mean_squared_error
import math
mse = mean_squared_error(y_val3, reg.predict(x_val3))
print("MSE: %.4f" % mse)
print("RMSE: %.4f" %math.sqrt(mse))
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_val3, reg.predict(x_val3))
print("MAE: %.4f" % mae)

MSE: 379.2796
RMSE: 19.4751
MAE: 14.7297


In [0]:
x_val4 = X[y_val<100].iloc[:,:].values
y_val4 = Y[y_val<100].iloc[:,:].values

In [0]:
from sklearn.metrics import mean_squared_error
import math
mse = mean_squared_error(y_val4, reg.predict(x_val4))
print("MSE: %.4f" % mse)
print("RMSE: %.4f" %math.sqrt(mse))
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_val4, reg.predict(x_val4))
print("MAE: %.4f" % mae)

MSE: 329.5595
RMSE: 18.1538
MAE: 14.3314
